In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [3]:
file_path = ['../../R98.xlsx', '../../R99.xlsx',
             '../../R1401.xlsx', '../../R1400.xlsx',
             '../../U98.xlsx', '../../U99.xlsx',
             '../../U1401.xlsx', '../../U1400.xlsx']

# Map for accessinng dataFrames of each year easily
file_to_dfs_map = {}

for path in file_path: 
    
    file_identifier = path.split('/')[-1].split('.')[0]
    dataframes_list = []

    excel_file = pd.ExcelFile(path)
    sheetNames = excel_file.sheet_names
    

    for sheet in sheetNames:
        df_name = f'df_{sheet}'  # Create a dynamic DataFrame name
        globals()[df_name] = pd.read_excel(path, sheet_name=sheet)
        dataframes_list.append(df_name)
        print(f'{df_name} DataFrame created with shape: {globals()[df_name].shape}')
    
    file_to_dfs_map[file_identifier] = dataframes_list

df_df_R98Data DataFrame created with shape: (18430, 17)
df_df_R98P1 DataFrame created with shape: (64296, 10)
df_df_R98P2 DataFrame created with shape: (18430, 45)
df_df_R98P3S01 DataFrame created with shape: (516797, 6)
df_df_R98P3S02 DataFrame created with shape: (4175, 6)
df_df_R98P3S03 DataFrame created with shape: (22378, 4)
df_df_R98P3S04 DataFrame created with shape: (68767, 5)
df_df_R98P3S05 DataFrame created with shape: (87170, 4)
df_df_R98P3S06 DataFrame created with shape: (30600, 4)
df_df_R98P3S07 DataFrame created with shape: (35450, 4)
df_df_R98P3S08 DataFrame created with shape: (28627, 4)
df_df_R98P3S09 DataFrame created with shape: (10839, 4)
df_df_R98P3S10 DataFrame created with shape: (0, 4)
df_df_R98P3S11 DataFrame created with shape: (11269, 4)
df_df_R98P3S12 DataFrame created with shape: (67753, 4)
df_df_R98P3S13 DataFrame created with shape: (197127, 3)
df_df_R98P3S14 DataFrame created with shape: (4387, 4)
df_df_R98P4S01 DataFrame created with shape: (10820, 18)

In [3]:
# make new column for sheet 7 part 3 of datas which show the main category of transport cost
for file_name in file_to_dfs_map.keys():
    globals()[file_to_dfs_map[file_name][9]]['code_categorical'] = globals()[file_to_dfs_map[file_name][9]]['code'].astype('str').str[0:3]

In [4]:
temp = globals()[file_to_dfs_map['R98'][9]].groupby('code_categorical')['value'].sum().sort_index() / globals()[file_to_dfs_map['R98'][9]].groupby('code_categorical')['value'].count().sort_index()

In [5]:
temp.sort_values()

code_categorical
724    1.445736e+05
732    4.218367e+05
731    5.782468e+05
723    7.767321e+05
734    7.916667e+05
722    7.966246e+05
735    7.968333e+06
733    2.648000e+07
Name: value, dtype: float64

In [6]:
# 733: X200, 735: X80, 722, 734, 723: X8, 731: X6, 732: X4, 724: x:1.5

In [7]:
#adding label rurall or urban to dataFrame
# 1: Rurall
# 2: urban

for file_name in ['R98', 'R99', 'R1400', 'R1401']:
    globals()[file_to_dfs_map[file_name][9]]['R/U'] = 1

for file_name in ['U98', 'U99', 'U1400', 'U1401']:
    globals()[file_to_dfs_map[file_name][9]]['R/U'] = 0

In [60]:
for file_name in file_to_dfs_map.keys():
    print(file_name)
    dfs= []

    # suming the one-hot code of category of transport cost
    # reason: number of family members use this fuel
    temp_df = globals()[file_to_dfs_map[file_name][9]].copy()
    if temp_df['code_categorical'].isin(['735']).sum():
        print(1)
        temp_df = pd.get_dummies(temp_df, columns= ['code_categorical'])
        temp_df = temp_df.groupby(by= 'Address').agg({
        'value': 'sum',
        'R/U': 'first',
        'code_categorical_722': 'sum',
        'code_categorical_723': 'sum',
        'code_categorical_724': 'sum',
        'code_categorical_731': 'sum',
        'code_categorical_732': 'sum',
        'code_categorical_733': 'sum',
        'code_categorical_734': 'sum',
        'code_categorical_735': 'sum'
        }).reset_index()

        temp_df['code_categorical_722'] = temp_df['code_categorical_722'] * 8
        temp_df['code_categorical_723'] = temp_df['code_categorical_723'] * 7.9
        temp_df['code_categorical_724'] = temp_df['code_categorical_724'] * 1.5
        temp_df['code_categorical_731'] = temp_df['code_categorical_731'] * 6
        temp_df['code_categorical_732'] = temp_df['code_categorical_732'] * 4
        temp_df['code_categorical_733'] = temp_df['code_categorical_733'] * 200
        temp_df['code_categorical_734'] = temp_df['code_categorical_734'] * 7.7
        temp_df['code_categorical_734'] = temp_df['code_categorical_735'] * 80

    else:
        print(2)
        temp_df = pd.get_dummies(temp_df, columns= ['code_categorical'])
        temp_df = temp_df.groupby(by= 'Address').agg({
        'value': 'sum',
        'R/U': 'first',
        'code_categorical_722': 'sum',
        'code_categorical_723': 'sum',
        'code_categorical_724': 'sum',
        'code_categorical_731': 'sum',
        'code_categorical_732': 'sum',
        'code_categorical_733': 'sum',
        'code_categorical_734': 'sum'
        }).reset_index()

        temp_df['code_categorical_722'] = temp_df['code_categorical_722'] * 8
        temp_df['code_categorical_723'] = temp_df['code_categorical_723'] * 7.9
        temp_df['code_categorical_724'] = temp_df['code_categorical_724'] * 1.5
        temp_df['code_categorical_731'] = temp_df['code_categorical_731'] * 6
        temp_df['code_categorical_732'] = temp_df['code_categorical_732'] * 4
        temp_df['code_categorical_733'] = temp_df['code_categorical_733'] * 200
        temp_df['code_categorical_734'] = temp_df['code_categorical_734'] * 7.7

    
    dfs.append(temp_df)  


    # handling p2_ sum of members that use vehicle, motorcycle, bicycle
    temp = globals()[file_to_dfs_map[file_name][2]].groupby(by=['Address']).agg({
    'vehicle': 'sum',
    'motorcycle': 'sum'
    'bicycle': 'sum'
    }).reset_index().copy()
    dfs.append(temp)

    #encoding genders for calculating number of males in each family
    temp_df = globals()[file_to_dfs_map[file_name][1]].copy()
    temp_df['gender_encoded'] = temp_df['gender'].map({
        'Male': 1,
        'Female': 0
    })
    #encode job for calculating the number of persons that are employed in each family
    temp_df = pd.get_dummies(temp_df, columns= ['occupationalst'])
    # seperate the age of the members
    bins = [0, 20, 30, 50, float('inf')]
    labels = ['<20', '20-30', '30-50', '>50']

    temp_df['age_group'] = pd.cut(temp_df['age'], bins= bins, labels= labels)
    temp_df = pd.get_dummies(temp_df, columns= ['age_group'])

    temp = temp_df.groupby(by= ['Address']).agg({
        'gender_encoded': 'sum',
        'occupationalst_Housewife': 'sum',
        'occupationalst_IncomeWOJob': 'sum',
        'occupationalst_Other': 'sum',
        'occupationalst_Student': 'sum',
        'occupationalst_child': 'sum',
        'occupationalst_employed': 'sum',
        'age_group_<20': 'sum',
        'age_group_20-30': 'sum',
        'age_group_30-50': 'sum',
        'age_group_>50': 'sum'
    }).reset_index()

    temp['gender_encoded'] = temp['gender_encoded'] * 4

    dfs.append(temp)

    # adding season of family interview(used for dividing test and train data)
    temp_df = globals()[file_to_dfs_map[file_name][0]][['Address', 'Fasl']]
    dfs.append(temp_df)

    merged_df = reduce(lambda left, right: pd.merge(left, right, on= 'Address', how= 'outer'), dfs)

    merged_df.to_excel(f'{file_name} regress.xlsx')

R98
1
R99
1
R1401
1
R1400
2
U98
1
U99
1
U1401
1
U1400
1


In [61]:
file_path = ['R98 regress.xlsx', 'R99 regress.xlsx',
             'R1401 regress.xlsx', 'R1400 regress.xlsx',
             'U98 regress.xlsx', 'U99 regress.xlsx',
             'U1401 regress.xlsx', 'U1400 regress.xlsx']

# Map for accessinng dataFrames of each year easily
file_to_dfs_map2 = {}

for path in file_path: 
    
    file_identifier = path.split('/')[-1].split('.')[0]
    dataframes_list = []

    excel_file = pd.ExcelFile(path)
    sheetNames = excel_file.sheet_names
    

    for sheet in sheetNames:
        df_name = f'df_{sheet}{path}'  # Create a dynamic DataFrame name
        globals()[df_name] = pd.read_excel(path, sheet_name=sheet)
        dataframes_list.append(df_name)
        print(f'{df_name} DataFrame created with shape: {globals()[df_name].shape}')
    
    file_to_dfs_map2[file_identifier] = dataframes_list

df_Sheet1R98 regress.xlsx DataFrame created with shape: (18430, 26)
df_Sheet1R99 regress.xlsx DataFrame created with shape: (18251, 26)
df_Sheet1R1401 regress.xlsx DataFrame created with shape: (18384, 26)
df_Sheet1R1400 regress.xlsx DataFrame created with shape: (19452, 25)
df_Sheet1U98 regress.xlsx DataFrame created with shape: (19898, 26)
df_Sheet1U99 regress.xlsx DataFrame created with shape: (19306, 26)
df_Sheet1U1401 regress.xlsx DataFrame created with shape: (19567, 26)
df_Sheet1U1400 regress.xlsx DataFrame created with shape: (21516, 26)


In [47]:
file_to_dfs_map2

{'R98 regress': ['df_Sheet1R98 regress.xlsx'],
 'R99 regress': ['df_Sheet1R99 regress.xlsx'],
 'R1401 regress': ['df_Sheet1R1401 regress.xlsx'],
 'R1400 regress': ['df_Sheet1R1400 regress.xlsx'],
 'U98 regress': ['df_Sheet1U98 regress.xlsx'],
 'U99 regress': ['df_Sheet1U99 regress.xlsx'],
 'U1401 regress': ['df_Sheet1U1401 regress.xlsx'],
 'U1400 regress': ['df_Sheet1U1400 regress.xlsx']}

In [62]:
# prepare Test_data
R1401_df = globals()[file_to_dfs_map2['R1401 regress'][0]]
U1401_df = globals()[file_to_dfs_map2['U1401 regress'][0]]


R1401_winter = R1401_df[R1401_df['Fasl'] == 4]
U1401_winter = U1401_df[U1401_df['Fasl'] == 4]

test_data = pd.concat((R1401_winter, U1401_winter), axis= 0).reset_index()

In [49]:
test_data[pd.isna(test_data['value'])]

,index,Unnamed: 0,Address,value,R/U,code_categorical_722,code_categorical_723,code_categorical_724,code_categorical_731,code_categorical_732,...,occupationalst_IncomeWOJob,occupationalst_Other,occupationalst_Student,occupationalst_child,occupationalst_employed,age_group_<20,age_group_20-30,age_group_30-50,age_group_>50,Fasl
5,17,17,20001384141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,2,4
6,36,36,20001384525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,1,0,1,4
9,39,39,20001384533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,1,0,3,0,4
11,41,41,20001384541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,1,4
19,85,85,20001385330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,3,0,2,2,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9303,19031,19031,12908366038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,1,4
9304,19032,19032,12908366041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,2,4
9351,19229,19229,13001371028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,2,0,1,0,2,4
9380,19341,19341,13001375132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,1,4


In [63]:
R1401_not_winter = R1401_df[R1401_df['Fasl'] != 4]
U1401_not_winter = U1401_df[U1401_df['Fasl'] != 4]

In [64]:
dfs = []

for file_name in file_to_dfs_map2.keys():
    if file_name == 'R1401 regress':
        dfs.append(R1401_not_winter)
    elif file_name == 'U1401 regress':
        dfs.append(U1401_not_winter)
    else:
        dfs.append(globals()[file_to_dfs_map2[file_name][0]])

train_data = reduce(lambda left, right: pd.concat([left, right], axis= 0), dfs)

In [65]:
train_data['code_categorical_735'] = train_data['code_categorical_735'].fillna(0)

In [66]:
# Delete rows with nan value
# Reason: this addresses belongs to families with no address in sheet of transport

train_data_cleaned = train_data.dropna()
test_data_cleaned = test_data.dropna()

In [67]:
# Delete columns which doesn't help model

train_data_cleaned.drop(columns= ['Unnamed: 0', 'Address', 'Fasl'], inplace= True)
test_data_cleaned.drop(columns= ['Unnamed: 0', 'Address', 'Fasl', 'index'], inplace= True)

C:\Users\Poumi008\AppData\Local\Temp\ipykernel_10304\177336172.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_cleaned.drop(columns= ['Unnamed: 0', 'Address', 'Fasl'], inplace= True)
C:\Users\Poumi008\AppData\Local\Temp\ipykernel_10304\177336172.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_cleaned.drop(columns= ['Unnamed: 0', 'Address', 'Fasl', 'index'], inplace= True)


In [68]:
test_data_cleaned.reset_index(drop= True, inplace= True)
train_data_cleaned.reset_index(drop= True, inplace= True)

In [19]:
test_data_cleaned.to_excel('test.xlsx')
train_data_cleaned.to_excel('train.xlsx')

In [56]:
x = train_data_cleaned.drop(columns= ['value'])
y = train_data_cleaned['value']

In [61]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(x)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [21]:
x

,R/U,code_categorical_722,code_categorical_723,code_categorical_724,code_categorical_731,code_categorical_732,code_categorical_733,code_categorical_734,code_categorical_735,vehicle,motorcycle,gender_encoded,age_group_<20,age_group_20-30,age_group_30-50,age_group_>50
0,1.0,8.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0,1,3.0,1.0,1.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0,0,1.0,0.0,0.0,0.0,2.0
2,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1.0,0.0,0.0,0.0,2.0
3,1.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1.0,2.0,0.0,2.0,0.0
4,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0,0,4.0,3.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129132,0.0,8.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1,0,2.0,3.0,1.0,2.0,0.0
129133,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0,0,2.0,0.0,2.0,1.0,1.0
129134,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0,0,1.0,1.0,0.0,2.0,0.0
129135,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0,0,3.0,3.0,0.0,1.0,0.0


In [72]:
test_data_cleaned.drop(columns= ['index'], inplace= True)

C:\Users\Poumi008\AppData\Local\Temp\ipykernel_33384\1733286068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_cleaned.drop(columns= ['index'], inplace= True)


In [73]:
pca = PCA()
pcaFinal = PCA(n_components= d)
reduced_train = pcaFinal.fit_transform(x)
reduced_test = pcaFinal.transform(test_data_cleaned.drop(columns= ['value']))

In [75]:
reduced_train

array([[ 9.17886404e-01, -1.92238843e-01, -1.42346634e+00,
        -1.60622979e+00,  8.96838574e-01,  2.03382685e-03],
       [-7.70589188e+00, -1.13546787e-01,  1.08477921e+00,
         3.94095599e-02, -1.96147368e+00, -2.79550732e-02],
       [ 2.27580876e+00, -2.23606702e-01, -4.25568748e+00,
         9.47425232e-01, -2.11395925e+00, -1.89587872e-02],
       ...,
       [-6.09841252e+00, -1.50017908e-01, -1.69066778e+00,
         2.34300771e+00,  5.89999930e-01, -2.77311189e-02],
       [-7.48758300e+00, -1.18084461e-01,  1.11029508e+00,
        -1.24812564e-01,  2.55625721e+00, -4.44902169e-03],
       [-7.57743273e+00, -1.12889890e-01,  1.17390808e+00,
        -1.13129026e-01, -4.05796883e-01, -1.59626361e-02]])

In [63]:
x

,R/U,code_categorical_722,code_categorical_723,code_categorical_724,code_categorical_731,code_categorical_732,code_categorical_733,code_categorical_734,code_categorical_735,vehicle,...,occupationalst_Housewife,occupationalst_IncomeWOJob,occupationalst_Other,occupationalst_Student,occupationalst_child,occupationalst_employed,age_group_<20,age_group_20-30,age_group_30-50,age_group_>50
0,1.0,8.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0,...,1.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
3,1.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,...,1.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,2.0,0.0
4,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0,...,1.0,0.0,0.0,1.0,2.0,1.0,3.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129132,0.0,8.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1,...,0.0,0.0,0.0,2.0,2.0,1.0,3.0,1.0,2.0,0.0
129133,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0
129134,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,0.0
129135,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0,...,0.0,0.0,1.0,0.0,2.0,1.0,3.0,0.0,1.0,0.0


In [69]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(x, y, shuffle= True, test_size= 0.32)

In [70]:
from sklearn.ensemble import GradientBoostingRegressor
from skopt import BayesSearchCV

search_space = {
    'n_estimators': (50, 1000),
    'learning_rate': (0.01, 0.3, 'uniform'),
    'max_depth': (2, 10),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20),
    'subsample': (0.5, 1.0, 'uniform'),
    'max_features': ['sqrt', 'log2', None],
}

gbr = GradientBoostingRegressor(random_state= 88)

opt = BayesSearchCV(
    estimator=gbr,
    search_spaces=search_space,
    n_iter=5,  # Number of iterations
    scoring='r2',  # Optimize R2 score
    cv=4,  # 5-fold cross-validation
    random_state=88,
    verbose= 1,
    n_jobs=-1  # Use all available cores
)


opt.fit(X_train, y_train)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits


BayesSearchCV(cv=4, estimator=GradientBoostingRegressor(random_state=88),
              n_iter=5, n_jobs=-1, random_state=88, scoring='r2',
              search_spaces={'learning_rate': (0.01, 0.3, 'uniform'),
                             'max_depth': (2, 10),
                             'max_features': ['sqrt', 'log2', None],
                             'min_samples_leaf': (1, 20),
                             'min_samples_split': (2, 20),
                             'n_estimators': (50, 1000),
                             'subsample': (0.5, 1.0, 'uniform')},
              verbose=1)

In [71]:
from sklearn.metrics import r2_score

y_pred = opt.predict(X_valid)
r2_score(y_valid, y_pred)

0.24967459363577882

In [72]:
r2_score(test_data_cleaned['value'], opt.predict(test_data_cleaned.drop(columns=['value'])))

0.1302748093635132

In [73]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor

base_estimator = RandomForestRegressor(random_state=42)

# Initialize BaggingRegressor with the base_estimator
bagging = BaggingRegressor(estimator=base_estimator, random_state=42)

# Define the search space for hyperparameter tuning
search_space_bagging = {
    'n_estimators': (10, 100),                # Number of estimators in the bagging ensemble
    'max_samples': (0.5, 1.0, 'uniform'),    # Fraction of samples used per base estimator
    'max_features': (0.5, 1.0, 'uniform'),   # Fraction of features used per base estimator
    'estimator__n_estimators': (50, 200),     # Number of trees in RandomForest
    'estimator__max_depth': (3, 10),          # Max depth of RandomForest
    'estimator__min_samples_split': (2, 10),  # Min samples to split in RandomForest
}

# Set up Bayesian optimization for BaggingRegressor
opt_bagging = BayesSearchCV(
    estimator=bagging,
    search_spaces=search_space_bagging,
    n_iter=5,  # Number of iterations
    cv=5,       # 5-fold cross-validation
    scoring='r2',  # Optimize for R2 score
    random_state=88,
    verbose= 1,
    n_jobs=-1   # Use all available cores
)

# Perform the search and fit
opt_bagging.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [ ]:
y_pred_bagging = opt_bagging.predict(X_valid)
r2_bagging = r2_score(y_valid, y_pred_bagging)